In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset from your CSV file
dolar_file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv'
dolar_data = pd.read_csv(dolar_file_path)

# Convert the 'Date' column to datetime type
dolar_data['Date'] = pd.to_datetime(dolar_data['Date'])

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dolar_data.drop(['Date'], axis=1))

# Split into train and test sets based on date
train_data = scaled_data[dolar_data['Date'] < pd.to_datetime('2022-01-01')]
test_data = scaled_data[dolar_data['Date'] >= pd.to_datetime('2022-01-01')]

# Function to create the LSTM dataset
def create_lstm_dataset(data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(data) - look_back - 1):
        a = data[i:(i + look_back), 0:data.shape[1]-1]
        dataX.append(a)
        dataY.append(data[i + look_back, -1])
    return np.array(dataX), np.array(dataY)

# Generate the dataset for LSTM
look_back = 1  # This can be changed to the number of time steps you want to look back
X_train, y_train = create_lstm_dataset(train_data, look_back)
X_test, y_test = create_lstm_dataset(test_data, look_back)

# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions back to original scale
train_predict = scaler.inverse_transform(np.concatenate((train_data[:, :-1], train_predict), axis=1))[:, -1]
test_predict = scaler.inverse_transform(np.concatenate((test_data[:, :-1], test_predict), axis=1))[:, -1]
y_train = scaler.inverse_transform(train_data)[:, -1]
y_test = scaler.inverse_transform(test_data)[:, -1]

# Calculate metrics
train_mse = mean_squared_error(y_train[look_back:], train_predict[:-1])
train_mae = mean_absolute_error(y_train[look_back:], train_predict[:-1])
test_mse = mean_squared_error(y_test[look_back:], test_predict[:-1])
test_mae = mean_absolute_error(y_test[look_back:], test_predict[:-1])

print(f'Train Score: MSE = {train_mse:.2f}, MAE = {train_mae:.2f}')
print(f'Test Score: MSE = {test_mse:.2f}, MAE = {test_mae:.2f}')


Epoch 1/100
3756/3756 - 4s - loss: 0.0173 - 4s/epoch - 981us/step
Epoch 2/100
3756/3756 - 3s - loss: 0.0059 - 3s/epoch - 770us/step
Epoch 3/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 770us/step
Epoch 4/100
3756/3756 - 3s - loss: 0.0059 - 3s/epoch - 758us/step
Epoch 5/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 750us/step
Epoch 6/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 746us/step
Epoch 7/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 753us/step
Epoch 8/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 759us/step
Epoch 9/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 743us/step
Epoch 10/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 733us/step
Epoch 11/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 735us/step
Epoch 12/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 745us/step
Epoch 13/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 754us/step
Epoch 14/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 745us/step
Epoch 15/100
3756/3756 - 3s - loss: 0.0058 - 3s/epoch - 740us/step
Epoc

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 3758 and the array at index 1 has size 3756